1. 문서의 내용을 읽는다 (w. Document Loaders)
2. 문서를 쪼갠다
    1. 토큰 수 초과로 답변을 생성하지 못할 수 있고,
    2. 문서가 길면 답변 생성이 오래걸림
3. 임베딩 후 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [20]:
# %pip install --upgrade --quiet docx2txt langchain-community
# %pip install -qU langchain-text-splitters
# %pip install langchain-chroma
# %pip install --upgrade pip setuptools wheel
# %pip install -U langchain langchainhub --quiet
# %pip install --upgrade --quiet langchain-pinecone

In [21]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = Docx2txtLoader("./tax.docx")
# document = loader.load()

# print(document)
# len(document)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,  # 문서 쪼갤 때 하나의 chunk 가 가지는 토큰 수
    chunk_overlap = 200,  # 쪼갤 때, 별개로 짜르는 게 아니라, 겹치게 해서 split 
)

document_list = loader.load_and_split(text_splitter=text_splitter)
# print(document_list)

In [22]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv() # 환경변수 불러오기 

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model='text-embedding-3-large') 

In [23]:
# pinecone 적용 
import os 

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

# split 된 document 들이 임베딩돼서 Pinecone 으로 들어감 
database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [24]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

retriever = database.as_retriever(search_kwargs={'k': 4})
retriever.invoke(query)

[Document(id='3563a06c-e129-4901-8770-8cf4388f7a0c', metadata={'source': './tax.docx'}, page_content='가. 연금소득자의 나이에 따른 다음의 세율\n\n      \n\n    나. 삭제 <2014. 12. 23.>\n\n    다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n\n  5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 연금 실제 수령연차 및 연금외수령 원천징수세율의 구체적인 내용은 대통령령으로 정한다.\n\n    가. 연금 실제 수령연차가 10년 이하인 경우: 연금외수령 원천징수세율의 100분의 70\n\n    나. 연금 실제 수령연차가 10년을 초과하는 경우: 연금외수령 원천징수세율의 100분의 60\n\n  6. 기타소득에 대해서는 다음에 규정하는 세율. 다만, 제8호를 적용받는 경우는 제외한다.\n\n    가. 제14조제3항제8호라목 및 마목에 해당하는 소득금액이 3억원을 초과하는 경우 그 초과하는 분에 대해서는 100분의 30\n\n    나. 제21조제1항제18호 및 제21호에 따른 기타소득에 대해서는 100분의 15\n\n    다. 삭제 <2014. 12. 23.>\n\n    라. 그 밖의 기타소득에 대해서는 100분의 20\n\n  7. 퇴직소득에 대해서는 기본세율\n\n  8. 대통령령으로 정하는 봉사료에 대해서는 100분의 5\n\n  9. 대통령령으로 정하는 금융투자소득에 대해서는 100분의 20\n\n  ② 제1항에도 불구하고 다음 각 호의 이자소득 및 배당소득에 대해서는 다음 각 호에서 정하는 세율을 원천징수세율로 한다. <개정 2013. 1. 1., 2017. 12. 19., 2018. 12. 31., 2022. 12. 31.>\n\n  1. 「민사집행법」 제113조 및 같은 법 제142조에 따라 법

In [25]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [26]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

d:\Program\miniconda3\envs\study\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [27]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [28]:
# Retrieval -> Question -> Answer 
from langchain.chains import RetrievalQA

# QA chain 만들기 
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=retriever, 
    chain_type_kwargs={"prompt": prompt}
)

ai_message = qa_chain.invoke({"query": query})
print(ai_message["query"])
print(ai_message["result"])


연봉 5천만원인 직장인의 소득세는 얼마인가요?
죄송하지만 제공된 문맥에서는 연봉 5천만원인 직장인의 소득세를 계산하는 데 필요한 구체적인 세율 정보가 포함되어 있지 않습니다. 한국의 소득세율은 다양한 요인에 따라 달라질 수 있으므로, 정확한 계산을 위해서는 최신 세법 자료나 세무 전문가의 조언이 필요합니다.
